# Generating Class Feature Viz

In [1]:
import os
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from torchvision.models import resnet50
import torchvision
import random
import pickle
from torch.nn import Conv2d


# set root dirs
grey_dir = "/home/local/data/sophie/imagenet/output/grey"
base_dir = "/home/local/data/sophie/imagenet/output/base"
single_dir = "/home/local/data/sophie/imagenet/output/single"

# grey_dir = "/home/local/data/sophie/imagenet/output/grey/continued/"
# base_dir = "/home/local/data/sophie/imagenet/output/base/continued/"
# configure GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# fetch imagenet classes 
with open('/home/local/data/sophie/imagenet/imagenet_classes.pkl', 'rb') as f:
    clses = pickle.load(f)

# define data augmentations
img_size = 224
transforms = torchvision.transforms.Compose([
      torchvision.transforms.RandomCrop(img_size, padding=random.randint(0, 8)), # jitter
      torchvision.transforms.RandomRotation((-45, 45)), # rotate
      torchvision.transforms.RandomResizedCrop(img_size, scale=(0.9, 1.2), ratio=(1.0, 1.,0)) # scale
    ])

def convert_to_single_channel(model):
    """
    Modifies the first convolutional layer of a given model to accept single-channel input.
    
    Args:
        model (torch.nn.Module): The model to be modified.
        
    Returns:
        torch.nn.Module: The modified model with a single-channel input.
    """
    # Identify the first convolutional layer
    conv1 = None
    for name, layer in model.named_modules():
        if isinstance(layer, Conv2d):
            conv1 = layer
            conv1_name = name
            break
    
    if conv1 is None:
        raise ValueError("The model does not have a Conv2D layer.")
    
    # Create a new convolutional layer with the same parameters except for the input channels
    new_conv1 = Conv2d(
        in_channels=1,  # Change input channels to 1
        out_channels=conv1.out_channels,
        kernel_size=conv1.kernel_size,
        stride=conv1.stride,
        padding=conv1.padding,
        bias=conv1.bias is not None
    )
    
    # Replace the old conv1 layer with the new one
    def recursive_setattr(model, attr, value):
        attr_list = attr.split('.')
        for attr_name in attr_list[:-1]:
            model = getattr(model, attr_name)
        setattr(model, attr_list[-1], value)
    
    recursive_setattr(model, conv1_name, new_conv1)
    
    return model

def load_models(base_model_path,grey_model_path):
    # initalize models
    base_model = resnet50(pretrained=True)
    grey_model = resnet50(pretrained=True)

    # load weights
    base_weights = torch.load(base_model_path, map_location='cpu')
    # configure state dict
    new_state_dict = {}
    for k, v in base_weights['model'].items():
        k = k.replace("module.", "")
        new_state_dict[k] = v
    # load model with state dict
    base_model.load_state_dict(new_state_dict)
    # disable grad
    for param in base_model.parameters():
      param.requires_grad_(False)
    
    # load weights
    grey_weights = torch.load(grey_model_path, map_location='cpu')
    # configure state dict
    new_state_dict = {}
    for k, v in grey_weights['model'].items():
        k = k.replace("module.", "")
        new_state_dict[k] = v
    # load model with state dict
    grey_model.load_state_dict(new_state_dict)
    # disable grad
    for param in grey_model.parameters():
      param.requires_grad_(False)

    # send to GPU
    base_model.to(device)
    grey_model.to(device)
    # set as eval 
    base_model.eval()
    grey_model.eval()
    return base_model, grey_model

def load_models_3(base_model_path,grey_model_path, single_model_path):
    # initalize models
    base_model = resnet50(pretrained=True)
    grey_model = resnet50(pretrained=True)
    single_model = resnet50(pretrained=True)

    # load weights
    base_weights = torch.load(base_model_path, map_location='cpu')
    # configure state dict
    new_state_dict = {}
    for k, v in base_weights['model'].items():
        k = k.replace("module.", "")
        new_state_dict[k] = v
    # load model with state dict
    base_model.load_state_dict(new_state_dict)
    # disable grad
    for param in base_model.parameters():
      param.requires_grad_(False)
    
    # load weights
    grey_weights = torch.load(grey_model_path, map_location='cpu')
    # configure state dict
    new_state_dict = {}
    for k, v in grey_weights['model'].items():
        k = k.replace("module.", "")
        new_state_dict[k] = v
    # load model with state dict
    grey_model.load_state_dict(new_state_dict)
    # disable grad
    for param in grey_model.parameters():
      param.requires_grad_(False)
        
    single_model = convert_to_single_channel(single_model)
     # load weights
    single_weights = torch.load(single_model_path, map_location='cpu')
    # configure state dict
    new_state_dict = {}
    for k, v in single_weights['model'].items():
        k = k.replace("module.", "")
        new_state_dict[k] = v
    # load model with state dict
    single_model.load_state_dict(new_state_dict)
    # disable grad
    for param in single_model.parameters():
      param.requires_grad_(False)

    # send to GPU
    base_model.to(device)
    grey_model.to(device)
    single_model.to(device)
    # set as eval
    base_model.eval()
    grey_model.eval()
    single_model.eval()
    return base_model, grey_model, single_model
    
def visualize_neuron(model, layer, neuron_indices, img_shape=(3, 224, 224), iterations=30, lr=0.1, device='"cuda:0"', transforms=None):#, device='cuda', transforms=None):
    # Initialize the input image with requires_grad=True
    input_img = torch.randn(1, *img_shape, requires_grad=True, device=device)
    # input_img = torch.randn(1, *img_shape, requires_grad=True, device='cpu')
    optimizer = torch.optim.Adam([input_img], lr=lr)

    activations = None

    # Non-intrusive hook function that captures activations
    def non_intrusive_hook_fn(module, input, output):
        nonlocal activations
        activations = output  # Do not detach to keep the graph intact

    # Register the hook to capture the layer's output
    hook = layer.register_forward_hook(non_intrusive_hook_fn)

    for i in range(iterations):
        optimizer.zero_grad()

        # Apply transformations if provided
        if transforms is not None:
            trs_img = transforms(input_img)
        else:
            trs_img = input_img

        # Forward pass
        model(trs_img)

        # Ensure that neuron_indices is within bounds
        if neuron_indices >= activations.shape[1]:
            raise ValueError(f"neuron_indices {neuron_indices} is out of bounds for activations with shape {activations.shape}")

        # Loss calculation as in Method 2
        if activations.dim() == 2:
            channel_activations = activations[:, neuron_indices]
        else:
            channel_activations = activations[:, neuron_indices, :, :]

        loss = -channel_activations.mean()

        # Backward pass to calculate gradients
        loss.backward()
        optimizer.step()

    # Remove the hook
    hook.remove()

    return input_img.detach()

def generateClassImages(base_model, grey_model, neuron, its=3000, lr=0.05, layer_name='fc', seed=0,trs=transforms, device="cuda:0"):
    # get the target layers
    colour_layer1 = dict([*base_model.named_modules()])[layer_name]
    grey_layer1 = dict([*grey_model.named_modules()])[layer_name]
    # generate images
    colour_mean_img1 = visualize_neuron(base_model, colour_layer1, neuron, iterations=its, 
                                        lr=lr, transforms=trs, device=device)
    grey_mean_img1 = visualize_neuron(grey_model, grey_layer1, neuron, iterations=its, 
                                      lr=lr, transforms=trs, device=device)
    # normalise images
    colour_mean_img1_norm = (colour_mean_img1.cpu().numpy()-colour_mean_img1.cpu().numpy().min())/(colour_mean_img1.cpu().numpy().max()-colour_mean_img1.cpu().numpy().min())
    grey_mean_img1_norm = (grey_mean_img1.cpu().numpy()-grey_mean_img1.cpu().numpy().min())/(grey_mean_img1.cpu().numpy().max()-grey_mean_img1.cpu().numpy().min())
    
    return colour_mean_img1, colour_mean_img1_norm, grey_mean_img1, grey_mean_img1_norm

def generateClassImages_3(base_model, grey_model, single_model, neuron, its=3000, lr=0.05, layer_name='fc', seed=0,trs=transforms, device="cuda:0"):
    # get the target layers
    colour_layer1 = dict([*base_model.named_modules()])[layer_name]
    grey_layer1 = dict([*grey_model.named_modules()])[layer_name]
    single_layer1 = dict([*single_model.named_modules()])[layer_name]
    # generate images
    colour_mean_img1 = visualize_neuron(base_model, colour_layer1, neuron, iterations=its, 
                                        lr=lr, transforms=trs, device=device)
    grey_mean_img1 = visualize_neuron(grey_model, grey_layer1, neuron, iterations=its, 
                                      lr=lr, transforms=trs, device=device)
    single_mean_img1 = visualize_neuron(single_model, single_layer1, neuron, img_shape=(1, 224, 224),iterations=its, 
                                      lr=lr, transforms=trs, device=device)
    # normalise images
    colour_mean_img1_norm = (colour_mean_img1.cpu().numpy()-colour_mean_img1.cpu().numpy().min())/(colour_mean_img1.cpu().numpy().max()-colour_mean_img1.cpu().numpy().min())
    grey_mean_img1_norm = (grey_mean_img1.cpu().numpy()-grey_mean_img1.cpu().numpy().min())/(grey_mean_img1.cpu().numpy().max()-grey_mean_img1.cpu().numpy().min())
    single_mean_img1_norm = (single_mean_img1.cpu().numpy()-single_mean_img1.cpu().numpy().min())/(single_mean_img1.cpu().numpy().max()-single_mean_img1.cpu().numpy().min())
    
    return colour_mean_img1, colour_mean_img1_norm, grey_mean_img1, grey_mean_img1_norm, single_mean_img1, single_mean_img1_norm
    

#### Pizza

In [ ]:
neuron = 963 #pizza
img_class = clses[neuron].split(",")[0]
plt_dir = "/home/local/data/sophie/imagenet/output/viz_plots/{}".format(img_class)
if not os.path.exists(plt_dir):
    os.makedirs(plt_dir)

# iterate over models
for model in os.listdir(grey_dir):
    if "model" in model:
        # set current model paths
        grey_model_path = os.path.join(grey_dir, model)
        base_model_path = os.path.join(base_dir, model.replace("grey", "base"))
        # load models
        base_model, grey_model = load_models(base_model_path,grey_model_path)
        
        colour_image, norm_colour_image, grey_image, norm_grey_image, = generateClassImages(base_model, grey_model, neuron)
        
        # generate plot
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        plt.imshow(norm_colour_image[0].transpose(1,2,0))
        plt.axis('off')
        plt.title("Colour FC: {}".format(img_class))
        plt.subplot(1,2,2)
        plt.imshow(norm_grey_image[0].transpose(1,2,0))
        plt.axis('off')
        plt.title("Grey FC: {}".format(img_class))
        plt.savefig(os.path.join(plt_dir, "{}.png".format(model.split("_")[-1].split(".")[0])), bbox_inches='tight')

/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/jovyan/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 89.9MB/s]


In [ ]:
neuron = 963 #pizza
img_class = clses[neuron].split(",")[0]
plt_dir = "/home/local/data/sophie/imagenet/output/viz_plots/single/{}".format(img_class)
if not os.path.exists(plt_dir):
    os.makedirs(plt_dir)

# iterate over models
for model in os.listdir(single_dir):
    if "model" in model:
        # set current model paths
        single_model_path = os.path.join(single_dir, model)
        base_model_path = os.path.join(base_dir, model.replace("single", "base"))
        grey_model_path = os.path.join(grey_dir, model.replace("single", "grey"))
        # load models
        base_model, grey_model, single_model = load_models_3(base_model_path,grey_model_path, single_model_path)
        
        colour_image, norm_colour_image, grey_image, norm_grey_image, single_image, norm_single_image, = generateClassImages_3(base_model, grey_model, single_model, neuron, its=3000)
        
        # generate plot
        plt.figure(figsize=(15,5))
        plt.subplot(1,3,1)
        plt.imshow(norm_colour_image[0].transpose(1,2,0))
        plt.axis('off')
        plt.title("Colour FC: {}".format(img_class))
        plt.subplot(1,3,2)
        plt.imshow(norm_grey_image[0].transpose(1,2,0))
        plt.axis('off')
        plt.title("Grey FC: {}".format(img_class))
        plt.subplot(1,3,3)
        plt.imshow(norm_single_image[0].transpose(1,2,0), cmap="binary")
        plt.axis('off')
        plt.title("Single FC: {}".format(img_class))
        plt.savefig(os.path.join(plt_dir, "{}.png".format(model.split("_")[-1].split(".")[0])), bbox_inches='tight')

### 

In [ ]:
neuron = 388
img_class = clses[neuron].split(",")[0]
plt_dir = "/home/local/data/sophie/imagenet/output/viz_plots/{}".format(img_class)
if not os.path.exists(plt_dir):
    os.makedirs(plt_dir)

# iterate over models
for model in os.listdir(grey_dir):
    if "model" in model:
        # set current model paths
        grey_model_path = os.path.join(grey_dir, model)
        base_model_path = os.path.join(base_dir, model.replace("grey", "base"))
        # load models
        base_model, grey_model = load_models(base_model_path,grey_model_path)
        
        colour_image, norm_colour_image, grey_image, norm_grey_image, = generateClassImages(base_model, grey_model, neuron)
        
        # generate plot
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        plt.imshow(norm_colour_image[0].transpose(1,2,0))
        plt.axis('off')
        plt.title("Colour FC: {}".format(img_class))
        plt.subplot(1,2,2)
        plt.imshow(norm_grey_image[0].transpose(1,2,0))
        plt.axis('off')
        plt.title("Grey FC: {}".format(img_class))
        plt.savefig(os.path.join(plt_dir, "{}.png".format(model.split("_")[-1].split(".")[0])), bbox_inches='tight')

#### 

In [ ]:
neuron = 470
img_class = clses[neuron].split(",")[0]
plt_dir = "/home/local/data/sophie/imagenet/output/viz_plots/{}".format(img_class)
if not os.path.exists(plt_dir):
    os.makedirs(plt_dir)

# iterate over models
for model in os.listdir(grey_dir):
    if "model" in model:
        # set current model paths
        grey_model_path = os.path.join(grey_dir, model)
        base_model_path = os.path.join(base_dir, model.replace("grey", "base"))
        # load models
        base_model, grey_model = load_models(base_model_path,grey_model_path)
        
        colour_image, norm_colour_image, grey_image, norm_grey_image, = generateClassImages(base_model, grey_model, neuron)
        
        # generate plot
        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        plt.imshow(norm_colour_image[0].transpose(1,2,0))
        plt.axis('off')
        plt.title("Colour FC: {}".format(img_class))
        plt.subplot(1,2,2)
        plt.imshow(norm_grey_image[0].transpose(1,2,0))
        plt.axis('off')
        plt.title("Grey FC: {}".format(img_class))
        plt.savefig(os.path.join(plt_dir, "{}.png".format(model.split("_")[-1].split(".")[0])), bbox_inches='tight')

#### 

In [ ]:
neuron = 8
img_class = clses[neuron].split(",")[0]
plt_dir = "/home/local/data/sophie/imagenet/output/viz_plots/{}".format(img_class)
if not os.path.exists(plt_dir):
    os.makedirs(plt_dir)

# iterate over models
for model in os.listdir(grey_dir):
    # set current model paths
    grey_model_path = os.path.join(grey_dir, model)
    base_model_path = os.path.join(base_dir, model.replace("grey", "base"))
    # load models
    base_model, grey_model = load_models(base_model_path,grey_model_path)
    
    colour_image, norm_colour_image, grey_image, norm_grey_image, = generateClassImages(base_model, grey_model, neuron)
    
    # generate plot
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(norm_colour_image[0].transpose(1,2,0))
    plt.axis('off')
    plt.title("Colour FC: {}".format(img_class))
    plt.subplot(1,2,2)
    plt.imshow(norm_grey_image[0].transpose(1,2,0))
    plt.axis('off')
    plt.title("Grey FC: {}".format(img_class))
    plt.savefig(os.path.join(plt_dir, "{}.png".format(model.split("_")[-1].split(".")[0])), bbox_inches='tight')